In [1]:
import pyspark
import pandas as pd
#import datefinder
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import re   
import seaborn as sns
import statsmodels.api as sm
#from statsmodels.tsa.arima_model import ARIMA, ARIMAResults
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime
from fbprophet import Prophet
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

Importing plotly failed. Interactive plots will not work.


In [2]:
sc = pyspark.SparkContext(appName="Spark RDD")

In [3]:
import requests

CLEANING THE DATA

In [4]:
#Remove tuple with len !=14
def RemoveStrangeTupleLen(tup):
    if len(tup)==14:
        return tup
    
#CLEANING MAC_ADDRESS_SRC FIELD
def RemoveQuoteSrc(tup):
    if tup[0][0]=='"':
        tup[0]=tup[0][1:]
    return tup

def RemoveWeirdAddress(tup):
    if len(tup[0])>17:
        tup[0]=None
    return tup

#CLEANING CAT FIELD
def CATToInt(tup): 
    tup[1] = int(tup[1])
    return tup

#CLEANING TID FIELD
def replaceNullValue_TID(tup):
    if tup[2] != '' and tup[2] != None and tup[2] != 'NaN': 
        return tup
    else: 
        tup[2] = ''
        return tup
    
#CLEANING TS FIELD
def TSToFloat(tup): 
    tup[3] = float(round(float(tup[3])))
    return tup

#CLEANING DST FIELD
#None

#CLEANING SAC FIELD
def replaceNullValue_SAC(tup):
    if tup[5] != '' and tup[5] != None and tup[5] != 'NaN': 
        tup[5] = float(tup[5])
        return tup
    else: 
        tup[5] = None
        return tup

#CLEANING SIC FIELD
def replaceNullValue_SIC(tup):
    if tup[6] != '' and tup[6] != None and tup[6] != 'NaN': 
        tup[6] = float(tup[6])
        return tup
    else: 
        tup[6] = None
        return tup
    
#CLEANING ToD FIELD
def replaceNullValue_ToD(tup):
    if tup[7] != '' and tup[7] != None and tup[7] != 'NaN': 
        tup[7] = float(tup[7])
        return tup
    else: 
        tup[7] = None
        return tup

#CLEANING TN FIELD
def replaceNullValue_TN(tup):
    if tup[8] != '' and tup[8] != None and tup[8] != 'NaN': 
        tup[8] = float(tup[8])
        return tup
    else: 
        tup[8] = None
        return tup

#CLEANING THETA FIELD
def replaceNullValue_THETA(tup):
    if tup[9] != '' and tup[9] != None and tup[9] != 'NaN': 
        tup[9] = float(tup[9])
        return tup
    else: 
        tup[9] = None
        return tup

#CLEANING RHO FIELD
def replaceNullValue_RHO(tup):
    if tup[10] != '' and tup[10] != None and tup[10] != 'NaN': 
        tup[10] = float(tup[10])
        return tup
    else: 
        tup[10] = None
        return tup
    
#CLEANING FL FIELD 
def replaceNullValue_FL(tup):
    if tup[11] != '' and tup[11] != None and tup[11] != 'NaN': 
        tup[11] = float(tup[11])
        return tup
    else: 
        tup[11] = None
        return tup
    
#CLEANING CGS FIELD
def replaceNullValue_CGS(tup):
    if tup[12] != '' and tup[12] != None and tup[12] != 'NaN': 
        tup[12] = float(tup[12])
        return tup
    else: 
        tup[12] = None
        return tup
    
#CLEANING CHdg FIELD
    
def replaceNullValue_CHdg(tup):
    if tup[13] != '' and tup[13] != None and tup[13] != 'NaN': 
        tup[13] = float(tup[13])
        return tup
    else: 
        tup[13] = None
        return tup
    
def RemoveQuoteCHdg(tup):
    if tup[13][-1]=='"':
        tup[13]=tup[13][:-1]
    return tup


def cleaning(tup):
    tup = RemoveQuoteSrc(tup)
    tup = RemoveWeirdAddress(tup)
    tup = CATToInt(tup)
    tup = replaceNullValue_TID(tup)
    tup = TSToFloat(tup)
    tup = replaceNullValue_SAC(tup)
    tup = replaceNullValue_SIC(tup)
    tup = replaceNullValue_ToD(tup)
    tup = replaceNullValue_TN(tup) 
    tup = replaceNullValue_THETA(tup)
    tup = replaceNullValue_RHO(tup)
    tup = replaceNullValue_FL(tup)
    tup = replaceNullValue_CGS(tup)
    tup = replaceNullValue_CHdg(RemoveQuoteCHdg(tup))
    return tup
    
def main_clean(rdd):
    
    #header = rdd.first()
    #rdd = rdd.filter(lambda line: line != header)
    #rdd = rdd.map(lambda tup: RemoveStrangeTupleLen(tup))\
    #         .filter(lambda tup: tup!=None)
             
    rdd = rdd.map(lambda tup: cleaning(tup))
    return(rdd)

In [ ]:
'''
def cleaning(tup):
    if tup[0][0]=='"':tup[0]=tup[0][1:]        
    
    if len(tup[0])>17:tup[0]=None
    
    tup[1] = int(tup[1])

    if tup[2] != '' and tup[2] != None and tup[2] != 'NaN': None
    else: tup[2] = ''
     
    tup[3] = float(round(float(tup[3])))

    if tup[5] != '' and tup[5] != None and tup[5] != 'NaN': tup[5] = float(tup[5])    
    else: tup[5] = None

    if tup[6] != '' and tup[6] != None and tup[6] != 'NaN': tup[6] = float(tup[6])
    else: tup[6] = None
    
    if tup[7] != '' and tup[7] != None and tup[7] != 'NaN': tup[7] = float(tup[7])
    else: tup[7] = None

    if tup[8] != '' and tup[8] != None and tup[8] != 'NaN': tup[8] = float(tup[8])
    else: tup[8] = None

    if tup[9] != '' and tup[9] != None and tup[9] != 'NaN': tup[9] = float(tup[9])
    else: tup[9] = None

    if tup[10] != '' and tup[10] != None and tup[10] != 'NaN': tup[10] = float(tup[10])
    else: tup[10] = None
    
    if tup[11] != '' and tup[11] != None and tup[11] != 'NaN': tup[11] = float(tup[11])
    else: tup[11] = None

    if tup[12] != '' and tup[12] != None and tup[12] != 'NaN': tup[12] = float(tup[12])
    else: tup[12] = None

    if tup[13] != '' and tup[13] != None and tup[13] != 'NaN': tup[13] = float(tup[13])
    else: tup[13] = None
    
    if tup[13][-1]=='"': tup[13]=tup[13][:-1]
    
    return tup
    
def main_clean(rdd):
    rdd = rdd.map(lambda tup: cleaning(tup))
    return(rdd)
'''

INSERT IN THE DB

In [5]:
# we first need to import types (e.g. StructType, StructField, IntegerType, etc.)
from pyspark.sql.types import *

def getlistavion(): 
    QUERY = 'SELECT DISTINCT(TID) FROM global_temp.traffic'
    return spark.sql(QUERY).toPandas()

def main_db(rdd): 
    global traffic_df_explicit, trafficSchema, spark  
    
    traffic_df_explicit_aux = spark.createDataFrame(rdd, trafficSchema)
    traffic_df_explicit = traffic_df_explicit.unionAll(traffic_df_explicit_aux)       
    
    traffic_df_explicit.createOrReplaceGlobalTempView('traffic')
    
    traffic_df_explicit.cache()
    
    #Queries
    #print(getlistavion())
    #spark.sql("select * from global_temp.traffic").show()

#main_db(rdd_traffic_clean)    

In [ ]:
#spark.sql("select * from global_temp.traffic").show()

In [24]:
def getPacketsByPlane(tid):
    TID = '\'%' + tid + '%\'' 
    QUERY = 'SELECT * FROM global_temp.traffic \
            WHERE TID LIKE ' + str(TID)
    #print(QUERY)
    return spark.sql(QUERY).toPandas()

def dictRadarsByAvion(TID):
    df_temp = getPacketsByPlane(TID)
    list_radar = list(df_temp.groupby('DST').size().index)
    dictRadar = {}
    for dst in list_radar:
        #dictRadar[dst] = filterByPlaneAndRadar(TID, dst) ##TOO MUCH TIME
        dictRadar[dst] = df_temp[df_temp['DST'] == dst] 
    return dictRadar

import colorsys
import random
colorsys.hsv_to_rgb(359,100,100)
def randomColor(i, m):
    return rgb_to_hex(colorsys.hsv_to_rgb(float(i) / float(m), 1, 1))
def rgb_to_hex(rgb):
    rgbInt = (int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255))
    return '#%02x%02x%02x' % rgbInt

def plotRadar(dictRadar, var=None):
        if var == None:
            for var in list(dictRadar[list(dictRadar.keys())[0]].columns):
                i, m = 0, len(dictRadar.keys())
                for dst in dictRadar.keys():
                    plt.plot(dictRadar[dst]['TS'], dictRadar[dst][var], randomColor(i, m))
                    i = i + 1
                plt.xlabel('time(s)')
                plt.ylabel(var)
                plt.show()            
        else:
            i, m = 0, len(dictRadar.keys())
            for dst in dictRadar.keys():
                plt.plot(dictRadar[dst]['TS'], dictRadar[dst][var], randomColor(i, m))
                i = i + 1
            plt.xlabel('time(s)')
            plt.ylabel(var)
            plt.show()  
        
        
def viz(dictRadar, TID):
    
    plotRadar(dictRadar)
    
    sns.pairplot(data=getPacketsByPlane(TID), vars=['SAC', 'SIC', 'ToD', 'TN', 'THETA', 'RHO', 'FL', 'CGS', 'CHdg'])
    plt.show()
    
    sns.heatmap(data=getPacketsByPlane(TID)[['SIC', 'ToD', 'TN', 'THETA', 'FL', 'CGS', 'CHdg']].corr(), annot = True, cmap = 'Reds')
    plt.show()

In [28]:
def filterByPlaneAndRadar(tid, dst):
    TID = '\'%' + tid + '%\'' 
    DST = '\'%' + dst + '%\'' 
    QUERY = 'SELECT * FROM global_temp.traffic WHERE TID LIKE ' + str(TID) + ' AND DST LIKE ' +  str(DST) 
    return spark.sql(QUERY).toPandas()

def plotOneRadarForOnePlane(df_radar_avion):
    for var in df_radar_avion.columns:
        plt.plot(df_radar_avion['TS'], df_radar_avion[var])
        plt.xlabel('time(s)')
        plt.ylabel(var)
        plt.show() 
        
def viz(df_radar_avion):
    
    plotOneRadarForOnePlane(df_radar_avion)
    
    sns.pairplot(data=df_radar_avion, vars=['SAC', 'SIC', 'ToD', 'TN', 'THETA', 'RHO', 'FL', 'CGS', 'CHdg'])
    plt.show()
    
    sns.heatmap(data=df_radar_avion[['SIC', 'ToD', 'TN', 'THETA', 'FL', 'CGS', 'CHdg']].corr(), annot = True, cmap = 'Reds')
    plt.show()

In [29]:
def timeseries(df, var): 

    df_time_series_radar_avion = df.copy()
    df_time_series_radar_avion['TS_DATE'] = df_time_series_radar_avion['TS'].copy()
    
    for i in range(len(df_time_series_radar_avion['TS'])):
        df_time_series_radar_avion['TS_DATE'][i] = datetime.datetime.fromtimestamp(df_time_series_radar_avion['TS'][i].astype('int64'))
    
    ts = pd.Series(df_time_series_radar_avion[var].values, index = df_time_series_radar_avion['TS_DATE'])
    ts.plot(color='r')
    plt.title('Time Series Plot for CGS Variable over time')
    plt.show()
    
    result = seasonal_decompose(ts, model='additive', period=10)
    result.plot()
    plt.show()

    window = 20
    ma = ts.rolling(window).mean()
    mstd = ts.rolling(window).std()

    plt.figure()
    plt.plot(ts.index, ts, 'r')
    plt.plot(ma.index, ma, 'b')
    plt.fill_between(mstd.index, ma - 2 * mstd, ma + 2 * mstd, color='b', alpha=0.2)
    
    ###Fbprophet###
    
    m = Prophet()
    m.fit(df_time_series_radar_avion[['TS_DATE',var]].rename(columns={'TS_DATE': 'ds', var: "y"}))
    future = m.make_future_dataframe(periods=20,freq='min')
    future.tail()
    forecast = m.predict(future)
    m.plot(forecast)
    #m.plot_components(forecast)
    
    ##ARIMA/ARMA Models 
    
    ts_diff = ts - ts.shift()
    diff = ts_diff.dropna()
    plt.figure()
    plt.plot(diff)
    plt.title('First Difference Time Series Plot')
    plt.show()
    mod = ARIMA(ts, order = (1, 1, 1), seasonal_order = (2,0,0,12))
    results = mod.fit()
    ts.plot()
    plt.plot(results.fittedvalues[20:], color='red')
    plt.show()
    
    predVals = results.predict(len(ts.index), len(ts.index) + 200, typ='levels')
    dict_for = {}
    
    temps = df_time_series_radar_avion['TS'][len(ts.index) - 1]
    for i in range(len(ts.index), len(ts.index) + 200):
        dict_for[datetime.datetime.fromtimestamp(int(temps))] = predVals[i]
        temps = temps + round(df_time_series_radar_avion['TS'][len(ts.index) - 1] - df_time_series_radar_avion['TS'][len(ts.index) - 2])
    
    ts.plot()
    plt.plot(results.fittedvalues[20:], color='red')
    plt.plot(pd.Series(dict_for), color='orange')
    plt.show()
    
    window_for = 20
    ma_for = pd.Series(dict_for).rolling(window).mean()
    mstd_for = pd.Series(dict_for).rolling(window).std()

    plt.figure()
    plt.plot(ts.index, ts, 'b')
    plt.plot(results.fittedvalues[20:], color='red')
    plt.plot(pd.Series(dict_for), color='orange')
    plt.plot(ma_for.index, ma_for, 'b')
    plt.fill_between(mstd_for.index, ma_for - 2 * mstd_for, ma_for + 2 * mstd_for, color='g', alpha=1)

In [35]:
spark = pyspark.sql.SparkSession.builder.appName("Spark-Dataframe-SQL").getOrCreate()

trafficSchema = StructType ( [StructField("SRC", StringType(), True),
                                 StructField("CAT", LongType(), True),
                                 StructField("TID", StringType(), True),
                                 StructField("TS", DoubleType(), True),
                                 StructField("DST", StringType(), True),
                                 StructField("SAC", DoubleType(), True),
                                 StructField("SIC", DoubleType(), True),
                                 StructField("ToD", DoubleType(), True),
                                 StructField("TN", DoubleType(), True),
                                 StructField("THETA", DoubleType(), True),
                                 StructField("RHO", DoubleType(), True),
                                 StructField("FL", DoubleType(), True),
                                 StructField("CGS", DoubleType(), True),
                                 StructField("CHdg", DoubleType(), True),
                             ] )
    
traffic_df_explicit = spark.createDataFrame(spark.sparkContext.emptyRDD(),trafficSchema)
traffic_df_explicit.createOrReplaceGlobalTempView('traffic')

response = requests.get('http://192.168.37.142:50005/stream/2019-05-04-12:00:00/2019-05-04-16:00:00', stream=True)
#ype(response)
i = 0

list_aux = [] 

for data in response.iter_lines():
    #print(data.decode("UTF-8"))  
    #print(str(data)[1:])
    i = i + 1
    #print(i)
    #print([data.decode("UTF-8").split(",")]) 
    #print(data.decode("UTF-8").split(","))
    
    list_aux.append(data.decode("UTF-8").split(","))
    
    if not(i % 10000):
        print(i)
        rdd_traffic = sc.parallelize(list_aux)
        rdd_traffic_clean = main_clean(rdd_traffic)
        
        #print(rdd_traffic_clean.collect())
        main_db(rdd_traffic_clean) 
        list_aux = []
        
        #timeseries(df, var)
        #timeseries(filterByPlaneAndRadar('TOM549','01:00:5e:50:00:26'), 'CGS')
        #FIN24Q
           
    #rdd_traffic = sc.parallelize([data.decode("UTF-8").split(",")])
    #rdd_traffic_clean = main_clean(rdd_traffic)
    #print(rdd_traffic_clean.collect())
    #main_db(rdd_traffic_clean) 
    
    #data viz 
    
    #if (i == 1000): 
    #    viz(dictRadarsByAvion('EXS29RP'), 'EXS29RP')
    
    #data time series 
    
    if (i==100000): break 

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [38]:
spark.sql("select count(distinct(TID)) from global_temp.traffic").toPandas()

,count(DISTINCT TID)
0,1128


In [ ]:
spark.sql("select TID, COUNT(*) from global_temp.traffic GROUP BY TID").toPandas()

In [ ]:
#dictRadarsByAvion('EXS29RP')

In [ ]:
#viz(dictRadarsByAvion('EZY51AQ'), 'EZY51AQ')

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [ ]:
# Create a local StreamingContext with two working thread and batch interval of 1 second
sc_stream = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc_stream, 1)

text data received over a TCP socket connection

In [ ]:
# Create a DStream that will connect to hostname:port, like localhost:9999
lines = ssc.socketTextStream("http://192.168.37.142", 50005)

In [ ]:
# Split each line into words
words = lines.flatMap(lambda line: line.split(" "))

In [ ]:
# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [ ]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

creating DStreams from files as input sources.

In [ ]:
import sys
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [ ]:
sc_stream = SparkContext(appName="PysparkStreaming")
ssc_stream = StreamingContext(sc_stream, 3)   #Streaming will execute in each 3 seconds

In [ ]:
lines = ssc_stream.textFileStream("test.txt")  #'log/ mean directory name

In [ ]:
lines = open("log/test.txt")
lines.read()

In [ ]:
counts = lines.flatMap(lambda line: line.split(" ")) \
        .map(lambda x: (x, 1)) \
        .reduceByKey(lambda a, b: a + b)
counts.pprint()
ssc_stream.start()
ssc_stream.awaitTermination()

In [ ]:
import sys
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

"""
This is use for create streaming of text from txt files that creating dynamically 
from files.py code. This spark streaming will execute in each 3 seconds and It'll
show number of words count from each files dynamically
"""


def main():
    sc = SparkContext(appName="PysparkStreaming")
    ssc = StreamingContext(sc, 3)   #Streaming will execute in each 3 seconds
    lines = ssc.textFileStream('log/')  #'log/ mean directory name
    counts = lines.flatMap(lambda line: line.split(" ")) \
        .map(lambda x: (x, 1)) \
        .reduceByKey(lambda a, b: a + b)
    counts.pprint()
    ssc.start()
    ssc.awaitTermination()


if __name__ == "__main__":
    main()

DataFrame and SQL Operations

In [ ]:
#A chaque insert dans la base, une mise à jour des plots